
Loads and filters JSON, removing keys ending with "Comment."

Extracts Readback and Settings paths.

Identifies and deletes matching keys with identical values.

Saves the cleaned JSON without redundant data


In [2]:
import re
import json

def remove_keys_ending_with_comment(data):
    """Recursively removes all keys ending with 'Comment' from JSON structure."""
    if isinstance(data, dict):
        keys_to_delete = [key for key in data if key.endswith("Comment")]
        for key in keys_to_delete:
            del data[key]

        for key in list(data.keys()):  
            remove_keys_ending_with_comment(data[key])

    elif isinstance(data, list):
        for item in data:
            remove_keys_ending_with_comment(item)

def find_readback_and_settings_paths(data, prefix=""):
    """Recursively find all paths for Readback and Settings in a JSON structure."""
    readback_paths = {}
    settings_paths = {}

    for key, value in data.items():
        full_path = f"{prefix}/{key}" if prefix else key

        if isinstance(value, dict):
            sub_readback, sub_settings = find_readback_and_settings_paths(value, full_path)
            readback_paths.update(sub_readback)
            settings_paths.update(sub_settings)
        else:
            if "Readback" in full_path:
                readback_paths[full_path] = value
            elif "Settings" in full_path:
                settings_paths[full_path] = value

    return readback_paths, settings_paths

def delete_key_by_full_path(data, full_path):
    """Deletes a key from the JSON structure using its full path."""
    keys = full_path.split("/")  
    current = data
    stack = []  

    for key in keys[:-1]:  
        if key in current and isinstance(current[key], dict):
            stack.append((current, key))  
            current = current[key]
        else:
            return  

    last_key = keys[-1]

    if last_key in current:
        del current[last_key]

    while stack:
        parent, key = stack.pop()
        if not parent[key]:  
            del parent[key]
        else:
            break  

def extract_common_part(full_key):
    """Extracts the key path without 'Readback' or 'Settings'."""
    pattern = re.compile(r'^(.*?)/?(Readback|Settings)/?(.*)$')
    match = pattern.match(full_key)
    if match:
        before = match.group(1)
        after = match.group(3)

        if before and after:
            return f"{before}/{after}"
        elif before:
            return before  
        else:
            return after  

    return full_key  

def find_and_delete_common_keys(readback_dict, settings_dict, json_data):
    """Finds and removes identical Readback & Settings values from JSON."""
    rb_mapping = {extract_common_part(k): k for k in readback_dict}
    st_mapping = {extract_common_part(k): k for k in settings_dict}

    common_parts = set(rb_mapping.keys()) & set(st_mapping.keys())

    for common in common_parts:
        rb_key = rb_mapping[common]
        st_key = st_mapping[common]

        if readback_dict[rb_key] == settings_dict[st_key]:
            print(f"Deleting identical keys for '{common}':")
            print(f"    Readback key: {readback_dict[rb_key]}")
            print(f"    Settings key: {settings_dict[st_key]}")

            delete_key_by_full_path(json_data, rb_key)
            delete_key_by_full_path(json_data, st_key)

    return json_data




In [3]:
def read_json_from_file(file_path):
    with open(file_path, 'r') as file:
        return json.load(file)
def save_json_to_file(data, filename):
    with open(filename, "w") as f:
        json.dump(data, f,separators=(',', ':'))

In [8]:
import re
import json

def remove_keys_ending_with_comment(data):
    """Recursively removes all keys ending with 'Comment', 'comment', 'Comments', or 'comments'."""
    if isinstance(data, dict):
        keys_to_delete = [key for key in data if re.search(r'(Comment|comment|Comments|comments)$', key)]
        for key in keys_to_delete:
            del data[key]

        for key in list(data.keys()):  
            remove_keys_ending_with_comment(data[key])

    elif isinstance(data, list):
        for item in data:
            remove_keys_ending_with_comment(item)

def find_readback_and_settings_paths(data, prefix=""):
    """Recursively find all paths for Readback and Settings in a JSON structure."""
    readback_paths = {}
    settings_paths = {}

    for key, value in data.items():
        full_path = f"{prefix}/{key}" if prefix else key

        if isinstance(value, dict):
            sub_readback, sub_settings = find_readback_and_settings_paths(value, full_path)
            readback_paths.update(sub_readback)
            settings_paths.update(sub_settings)
        else:
            if "Readback" in full_path:
                readback_paths[full_path] = value
            elif "Settings" in full_path:
                settings_paths[full_path] = value

    return readback_paths, settings_paths

def delete_key_by_full_path(data, full_path):
    """Deletes a key from the JSON structure using its full path."""
    keys = full_path.split("/")  
    current = data
    stack = []  

    for key in keys[:-1]:  
        if key in current and isinstance(current[key], dict):
            stack.append((current, key))  
            current = current[key]
        else:
            return  

    last_key = keys[-1]

    if last_key in current:
        del current[last_key]

    while stack:
        parent, key = stack.pop()
        if not parent[key]:  
            del parent[key]
        else:
            break  

def extract_common_part(full_key):
    """Extracts the key path without 'Readback' or 'Settings'."""
    pattern = re.compile(r'^(.*?)/?(Readback|Settings)/?(.*)$')
    match = pattern.match(full_key)
    if match:
        before = match.group(1)
        after = match.group(3)

        if before and after:
            return f"{before}/{after}"
        elif before:
            return before  
        else:
            return after  

    return full_key  

def find_and_delete_common_keys(readback_dict, settings_dict, json_data):
    """Finds and removes identical Readback & Settings values from JSON."""
    rb_mapping = {extract_common_part(k): k for k in readback_dict}
    st_mapping = {extract_common_part(k): k for k in settings_dict}

    common_parts = set(rb_mapping.keys()) & set(st_mapping.keys())

    for common in common_parts:
        rb_key = rb_mapping[common]
        st_key = st_mapping[common]

        if readback_dict[rb_key] == settings_dict[st_key]:
            print(f"Deleting identical keys for '{common}':")
            print(f"    Readback key: {readback_dict[rb_key]}")
            print(f"    Settings key: {settings_dict[st_key]}")

            delete_key_by_full_path(json_data, rb_key)
            delete_key_by_full_path(json_data, st_key)

    return json_data



In [9]:


json_data = read_json_from_file("24250227_135126_end_remove_system_keys_except_clients.json")
# Step 1: Remove "Comment" keys FIRST
filtered_json = json_data.copy()
remove_keys_ending_with_comment(filtered_json)

# Step 2: Extract Readback and Settings Paths from the filtered JSON
readback_paths, settings_paths = find_readback_and_settings_paths(filtered_json)

# Step 3: Process and clean up JSON without "Comment" keys
updated_json = find_and_delete_common_keys(readback_paths, settings_paths, filtered_json)

save_json_to_file(updated_json,"24250227_135126_end_filterreadback_Settings.json")


Deleting identical keys for 'Detectors/Det13/L1Trigger/DCRC0/LC Clipping Discard MSB':
    Readback key: ['0x00000000', '0x00000000', '0x00000000', '0x00000000']
    Settings key: ['0x00000000', '0x00000000', '0x00000000', '0x00000000']
Deleting identical keys for 'Detectors/Det06/Phonon/LockpointAdjust (mV)':
    Readback key: [-0.092286564, -0.15655756, -0.17550927, -0.0049439231, -0.090638593, -0.1153582, -0.060151063, -0.080750741, -0.063447013, -0.13430992, -0.029663539, -0.062623024]
    Settings key: [-0.092286564, -0.15655756, -0.17550927, -0.0049439231, -0.090638593, -0.1153582, -0.060151063, -0.080750741, -0.063447013, -0.13430992, -0.029663539, -0.062623024]
Deleting identical keys for 'Detectors/Det20/L1Trigger/DCRC0/FIR Coeff SubMod2':
    Readback key: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 